In [ ]:
import warnings
warnings.filterwarnings('ignore')

from validation_practical_functions_light import *


In [ ]:
lonmin=35
lonmax=41.5
latmin=-4.5
latmax=4.5
month_start=3
month_end=5

In [ ]:
wrsi_file='wrsi_hist_monthly_1983_2019.nc'
ndvi_file='ndvi_hist_monthly_1983_2015.nc'

wrsi,lons,lats=mask_3d_data(wrsi_file,'beta',lonmin,lonmax,latmin,latmax)
ndvi,lons,lats=mask_3d_data(ndvi_file,'ndvi',lonmin,lonmax,latmin,latmax)

wrsi_seas=extract_season(wrsi,month_start,month_end)
ndvi_seas=extract_season(ndvi,month_start,month_end)

#Calculate the time series of the areal mean masked data
wrsi_stand_anom=(np.nanmean(wrsi_seas,axis=(0,1))[:33]-np.nanmean(wrsi_seas[:,:,:33]))/np.nanstd(wrsi_seas[:,:,:33])
ndvi_stand_anom=(np.nanmean(ndvi_seas,axis=(0,1))[:33]-np.nanmean(ndvi_seas[:,:,:33]))/np.nanstd(ndvi_seas[:,:,:33])

#Calculate a spatially varying climatological seasonal mean
wrsi_pix=np.mean(wrsi_seas[:,:,0:33],axis=2)
ndvi_pix=np.mean(ndvi_seas,axis=2)


In [ ]:
output=map_corr_plot(wrsi_seas[:,:,0:33],ndvi_seas,'WRSI','NDVI',1,1,lons,lats,plot_out='myplot.png')



In [ ]:

output=spat_corr_plot(wrsi_pix, ndvi_pix, "WRSI", "NDVI",plot_out='myplot.png')


In [ ]:
corr_plot(wrsi_stand_anom, ndvi_stand_anom, "WRSI", "NDVI",plot_out='myplot.png')

In [ ]:

ts_plot(wrsi_stand_anom, ndvi_stand_anom, "WRSI", "NDVI",color1='Black',color2='Red',xtick_locs=np.arange(2,35,5),xtick_labs=np.arange(1985,2017,5),plot_out='myplot.png')

In [ ]:
hindcast_file='mam_wrsi_init_Mar01_2003_2016.nc'

#Get metadata

hindcast_metadata=pds.read_csv('hindcasts.csv')
single_file=hindcast_metadata[hindcast_metadata.iloc[:,0]==hindcast_file]

month_start=int(single_file['POI_start_month'].values)
month_end=int(single_file['POI_end_month'].values)

#Process hindcast data
ens_mean,lons,lats=mask_3d_data_ens(hindcast_file,'ens_mean',lonmin,lonmax,latmin,latmax)
ens_sd,lons,lats=mask_3d_data_ens(hindcast_file,'ens_sd',lonmin,lonmax,latmin,latmax)

ens_mean_ts=np.nanmean(ens_mean,axis=(0,1))
ens_sd_ts=np.nanmean(ens_sd,axis=(0,1))

#Calculate the time series of the areal mean masked data
ens_stand_anom=(np.nanmean(ens_mean[:,:,0:13],axis=(0,1))-np.nanmean(ens_mean[:,:,0:13]))/np.nanstd(ens_mean[:,:,0:13])

#Process historical data
wrsi_file='wrsi_hist_monthly_1983_2019.nc'
ndvi_file='ndvi_hist_monthly_1983_2015.nc'

#Mask wrsi and ndvi data on our shape file
wrsi,lons,lats =  mask_3d_data(wrsi_file,'beta',lonmin,lonmax,latmin,latmax)
ndvi,lons,lats =  mask_3d_data(ndvi_file,'ndvi',lonmin,lonmax,latmin,latmax)

#Extract period of interest and mask the resultant data on our shape file
wrsi_seas=extract_season(wrsi,month_start,month_end)
ndvi_seas=extract_season(ndvi,month_start,month_end)

#Calculate the time series of the areal mean masked data
wrsi_stand_anom=(np.nanmean(wrsi_seas[:,:,20:33],axis=(0,1))-np.nanmean(wrsi_seas[:,:,20:33]))/np.nanstd(wrsi_seas[:,:,20:33])
ndvi_stand_anom=(np.nanmean(ndvi_seas[:,:,20:33],axis=(0,1))-np.nanmean(ndvi_seas[:,:,20:33]))/np.nanstd(ndvi_seas[:,:,20:33])

wrsi_ts=np.nanmean(wrsi_seas,axis=(0,1))

In [ ]:
ts_plot(ndvi_stand_anom, ens_stand_anom, "Observed NDVI", "Hindcast WRSI",color1='Black',color2='Red',xtick_locs=np.arange(2,15,3),xtick_labs=np.arange(2003,2016,3),plot_out='myplot.png')

In [ ]:
output=map_corr_scatter_plot(ens_mean[:,:,0:13],ndvi_seas[:,:,20:33],'Hindcast WRSI anom','NDVI anom',1,1,lons,lats,plot_out='myplot.png')


In [ ]:

varin_seas=wrsi_seas[:,:,20:33]
ens_mean=ens_mean[:,:,0:13]
ens_sd=ens_sd[:,:,0:13]

percentile_in=5

output=roc_score(varin_seas,ens_mean,ens_sd,percentile_in,lons,lats,plot_out='myplot.png')